In [19]:
import ee
import geemap
from gee_0_utils import *

initialize()
config = ProjectConfig()

roi = config.roi
data_folder = config.data_folder
_, lulc, _ = desired_lulc()

biomass_raw = ee.Image(f"{data_folder}/raw/biomass")
proj = biomass_raw.projection().getInfo()

## Surrounding mature forests

- Mean biomass of surrounding mature forests (interpolated with Gaussian kernel)
- Mean fragmentation of surrounding mature forests (interpolated with Gaussian kernel)
- Total surrounding mature forest cover (normalized from 0-1)

In [20]:
def get_mature_mask(method, lulc):
    config = ProjectConfig()

    if method == 'eu':
        annual_changes = ee.ImageCollection('projects/JRC/TMF/v1_2023/AnnualChanges').mosaic().clip(config.roi)
        return annual_changes.select('Dec2020').eq(1).selfMask()
    elif method == 'mapbiomas':
        return lulc.eq(3).reduce(ee.Reducer.allNonZero()).selfMask()

def get_mature_biomass(method, biomass_raw, mature_mask):
    if method == 'eu':
        return biomass_raw.updateMask(mature_mask).rename("mature_biomass")
    elif method == 'mapbiomas':
        edge_detec = mature_mask.unmask(-1).zeroCrossing()
        distance_to_edge = edge_detec.fastDistanceTransform().sqrt() \
            .multiply(ee.Image.pixelArea().sqrt()).rename("distance_to_edge")
        distance_to_edge = distance_to_edge.gt(1000).selfMask().updateMask(mature_mask)
        return biomass_raw.updateMask(distance_to_edge).rename("mature_biomass")

def export_mature_biomass(method, export = True):

    biomass_raw = ee.Image(f"{data_folder}/raw/biomass")
    proj = biomass_raw.projection().getInfo()

    mature_mask = get_mature_mask(method, lulc)
    sur_cover = mature_mask.unmask(0).focalMean(radius=1000, units="meters").rename("sur_cover")
    mature_biomass = get_mature_biomass(method, biomass_raw, mature_mask)

    if export:
        export_image(mature_biomass, f"mature_biomass_{method}", folder = method, crsTransform = proj['transform'])
        export_image(mature_biomass, f"mature_biomass_500_{method}", folder = method, scale = 500)
        export_image(sur_cover, f"sur_cover_{method}", folder = method, scale = 30)

    return mature_biomass

# export_mature_biomass('eu')
# export_mature_biomass('mapbiomas')

In [21]:
def get_distance_to_nearest_mature(mature_biomass, method):

    secondary = ee.Image(f"{data_folder}/{method}/secondary")
    secondary = secondary.select("age")

    mature_biomass_vector = mature_biomass.reduceToVectors(geometry = roi, scale = 500, maxPixels = 1e13)
    distance_to_nearest_mature = mature_biomass_vector.distance(searchRadius = 500000).toInt().updateMask(secondary)

    export_image(distance_to_nearest_mature, f"distance_to_nearest_mature_{method}", folder = method, crsTransform = proj['transform'])

    return distance_to_nearest_mature

# Function to buffer and calculate mean
def buffer_distance(f):
    distance = ee.Number(f.get('distance'))
    buffered = ee.Feature(f.buffer(distance.add(2000)))
    return buffered

def export_nearest_mature_biomass(distance_to_mature_biomass, mature_biomass, method):
    categorical = ee.Image(f"{data_folder}/categorical")
    biome_mask = categorical.select('biome').inList([1, 4, 6])
    distance_to_mature_biomass = distance_to_mature_biomass \
        .addBands(biome_mask)

    distances_vector = distance_to_mature_biomass.stratifiedSample(
        numPoints = 50000, classBand = "biome", region = roi, geometries = True
    )
    
    distances_vector_buffer = distances_vector.map(buffer_distance)

    nearest_mature_biomass = mature_biomass.reduceRegions(distances_vector_buffer, ee.Reducer.firstNonNull(), 
                                            crs = proj['crs'], crsTransform = proj['transform'])

    # Export the mean_biomass_image as an asset
    task = ee.batch.Export.table.toAsset(
        collection = nearest_mature_biomass,
        assetId = f"{data_folder}/{method}/nearest_mature_biomass",
        description = "nearest_mature_biomass"
    )

    task.start()

mature_biomass = ee.Image(f"{data_folder}/eu/mature_biomass_500_eu")
distance_to_nearest_mature = get_distance_to_nearest_mature(mature_biomass, 'eu')
export_nearest_mature_biomass(distance_to_nearest_mature, mature_biomass, 'eu')

EEException: Image.load: Image asset 'projects/amazon-forest-regrowth/assets/eu/age_agbd' not found (does not exist or caller does not have access).

In [ ]:
nearest_mature_biomass = ee.FeatureCollection(f"{config.data_folder}/{method}/nearest_mature")
# change geometries from polygons to points
nearest_mature_biomass = nearest_mature_biomass.map(lambda f: ee.Feature(f.geometry().centroid()).copyProperties(f))

# Convert the FeatureCollection to an image
nearest_mature_biomass_image = nearest_mature_biomass.reduceToImage(
    properties = ['first'],
    reducer = ee.Reducer.first()
).toInt16().rename('nearest_mature_biomass')

export_image(nearest_mature_biomass_image, "nearest_mature_biomass_image", crsTransform = proj['transform'])

In [ ]:
gaus_kernel = ee.Kernel.gaussian(radius = 1000, sigma = 200, units = "meters")
mat_gaus_ker = (
    mature_biomass.focalMean(
        radius = 1000, units = "meters", kernel = gaus_kernel, iterations = 1
    )
    .rename("mat_biomass")
    .float()
)

# the fragmentation data also does not cover the entire landscape - it is targeted towards mature forests.
# frag file from Ma et al was originally reprojected in QGIS into WGS84 (original projection is not compatible with google earth engine)
# frag values vary from 0 to 1
frag = ee.Image(f"{data_folder}/raw/frag").rename("frag").clip(age.geometry())
frag_ker = frag.focalMean(radius = 5000, units = "meters").rename("frag")

# export_image(mat_gaus_ker, "mat_gaus_ker", scale = 30)
# export_image(frag_ker, "frag_ker", scale = 30)
